# Green Space Detection Pipeline

Multi-temporal analysis using Sentinel-2 satellite imagery and Random Forest classification.

## 1. Generate AOIs for 15 Cities

Create 512×512 pixel (5.12 km) square areas of interest centered on each city.

In [ ]:
import json
import os
import shapely.geometry
from shapely.ops import transform
import pyproj

In [ ]:
# City coordinates (longitude, latitude)
cities = {
    "Vienna": (16.3738, 48.2082), "Paris": (2.3522, 48.8566),
    "London": (-0.1276, 51.5074), "Toronto": (-79.3832, 43.6532),
    "Vancouver": (-123.1207, 49.2827), "San_Francisco": (-122.4194, 37.7749),
    "Lisbon": (-9.1393, 38.7223), "Madrid": (-3.7038, 40.4168),
    "Barcelona": (2.1734, 41.3851), "Berlin": (13.4050, 52.5200),
    "Amsterdam": (4.9041, 52.3676), "Melbourne": (144.9631, -37.8136),
    "Sydney": (151.2093, -33.8688), "Auckland": (174.7633, -36.8485),
    "Seattle": (-122.3321, 47.6062)
}

AOI_SIZE_M = 512 * 10  # 5.12 km square
OUTPUT_FOLDER = "aois_json"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
def create_square_aoi(center_lon, center_lat, size_m):
    """Create square AOI using UTM projection."""
    utm_zone = int((center_lon + 180) / 6) + 1
    utm = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')
    wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')

    project_to_utm = pyproj.Transformer.from_proj(wgs84, utm, always_xy=True).transform
    project_to_wgs = pyproj.Transformer.from_proj(utm, wgs84, always_xy=True).transform

    point = shapely.geometry.Point(center_lon, center_lat)
    point_utm = transform(project_to_utm, point)

    half_size = size_m / 2
    square_utm = shapely.geometry.box(
        point_utm.x - half_size, point_utm.y - half_size,
        point_utm.x + half_size, point_utm.y + half_size
    )

    return transform(project_to_wgs, square_utm)

# Generate GeoJSON for each city
for city, (lon, lat) in cities.items():
    square = create_square_aoi(lon, lat, AOI_SIZE_M)
    geojson = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "properties": {"city": city},
            "geometry": {"type": "Polygon", "coordinates": [list(square.exterior.coords)]}
        }]
    }
    
    filepath = os.path.join(OUTPUT_FOLDER, f"{city.replace(' ', '_')}.geojson")
    with open(filepath, "w") as f:
        json.dump(geojson, f, indent=2)
    print(f"✓ {city}")

print(f"\n✓ Saved {len(cities)} AOIs to '{OUTPUT_FOLDER}/'")

## 2. Process Sentinel-2 Multi-Temporal Stacks

Load and stack 4 bands (B02, B03, B04, B08) from 3 months (April, August, November).

In [ ]:
import rasterio
import numpy as np
from pathlib import Path

In [ ]:
# Configuration
BASE_PATH = "sentinel_data"
CITIES = ["London", "Melbourne", "Paris", "Seattle", "Stockholm"]

MONTHS = {
    "April": "April-10m",
    "August": "August-10m",
    "November": "November-10m"
}

BANDS = ["B02", "B03", "B04", "B08"]
OUTPUT_FOLDER = "processed_stacks"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
def find_band_file(folder_path, band_name):
    """Find band file with flexible pattern matching."""
    folder = Path(folder_path)
    if not folder.exists():
        return None
    
    patterns = [f"*_{band_name}_10m.jp2", f"*_{band_name}.jp2", 
                f"**/*_{band_name}_10m.jp2", f"**/IMG_DATA/**/*{band_name}*.jp2"]
    
    for pattern in patterns:
        matches = list(folder.glob(pattern))
        if matches:
            return str(matches[0])
    
    for file in folder.rglob("*.jp2"):
        if band_name in file.name:
            return str(file)
    return None

def load_month_stack(folder_path, bands):
    """Load and stack bands for one month."""
    band_arrays = []
    profile = None
    
    for band in bands:
        band_file = find_band_file(folder_path, band)
        if band_file is None:
            continue
            
        with rasterio.open(band_file) as src:
            band_arrays.append(src.read(1))
            if profile is None:
                profile = src.profile.copy()
    
    if not band_arrays:
        return None, None
    
    return np.stack(band_arrays, axis=0), profile

In [ ]:
for city in CITIES:
    print(f"\nProcessing {city}...")
    all_months = []
    final_profile = None
    
    for month_name, month_suffix in MONTHS.items():
        folder = f"{BASE_PATH}/{city}/{city}-{month_suffix}"
        if not os.path.exists(folder):
            print(f"  ⚠ Missing: {month_name}")
            continue
        
        month_data, profile = load_month_stack(folder, BANDS)
        if month_data is not None:
            all_months.append(month_data)
            if final_profile is None:
                final_profile = profile
            print(f"  ✓ {month_name}: {month_data.shape[0]} bands")
    
    if not all_months:
        print(f"  ✗ No data for {city}")
        continue
    
    # Create multi-month stack
    full_stack = np.concatenate(all_months, axis=0).astype(np.uint16)
    final_profile.update(count=full_stack.shape[0], dtype='uint16', compress='lzw')
    
    output_path = f"{OUTPUT_FOLDER}/{city}_MultiMonth_stack.tif"
    with rasterio.open(output_path, 'w', **final_profile) as dst:
        dst.write(full_stack)
    
    print(f"  ✓ Saved: {output_path} ({full_stack.shape[0]} bands)")

print(f"\n✓ Processing complete!")

## 3. Fetch OSM Green Space Data

Download ground truth labels from OpenStreetMap (parks, forests, gardens).

In [ ]:
import osmnx as ox
import geopandas as gpd

In [ ]:
# OSM green space tags
tags = {
    "leisure": ["park", "garden"],
    "landuse": ["forest", "grass", "meadow", "village_green"],
    "natural": ["wood", "scrub"]
}

cities = ["London", "Melbourne", "Paris", "Seattle", "Stockholm"]

for city in cities:
    aoi_file = f"aois_json/{city}.geojson"
    output_file = f"sentinel_data/{city}/{city}_OSM_green.geojson"
    
    if not os.path.exists(aoi_file):
        print(f"⚠ Missing AOI: {city}")
        continue
    
    try:
        aoi = gpd.read_file(aoi_file).to_crs("EPSG:4326")
        polygon = aoi.geometry.iloc[0]
        
        green_features = ox.features_from_polygon(polygon, tags)
        green_features = green_features[green_features.geometry.type.isin(["Polygon", "MultiPolygon"])]
        
        if len(green_features) == 0:
            print(f"⚠ {city}: No green features found")
            continue
        
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        green_features.to_file(output_file, driver="GeoJSON")
        
        area_km2 = green_features.to_crs('EPSG:3857').area.sum() / 1e6
        print(f"✓ {city}: {len(green_features)} features, {area_km2:.2f} km²")
        
    except Exception as e:
        print(f"✗ {city}: {str(e)}")

print("\n✓ OSM data fetched!")

## 4. Rasterize OSM Labels

Convert vector polygons to raster labels matching Sentinel-2 resolution.

In [ ]:
from rasterio.features import rasterize

cities = ["London", "Melbourne", "Paris", "Seattle", "Stockholm"]

for city in cities:
    stack_path = f"processed_stacks/{city}_MultiMonth_stack.tif"
    osm_path = f"sentinel_data/{city}/{city}_OSM_green.geojson"
    label_path = f"sentinel_data/{city}/{city}_OSM_labels.tif"
    
    if not os.path.exists(stack_path) or not os.path.exists(osm_path):
        print(f"⚠ Missing data for {city}")
        continue
    
    try:
        with rasterio.open(stack_path) as src:
            transform = src.transform
            out_shape = (src.height, src.width)
            crs = src.crs
        
        green_features = gpd.read_file(osm_path).to_crs(crs)
        green_features = green_features[green_features.geometry.type.isin(["Polygon", "MultiPolygon"])]
        
        labels = rasterize(
            [(geom, 1) for geom in green_features.geometry],
            out_shape=out_shape, transform=transform, fill=0,
            all_touched=True, dtype="uint8"
        )
        
        with rasterio.open(label_path, "w", driver="GTiff", height=out_shape[0],
                          width=out_shape[1], count=1, dtype="uint8", crs=crs,
                          transform=transform, compress="lzw") as dst:
            dst.write(labels, 1)
        
        green_pct = 100 * np.sum(labels == 1) / labels.size
        print(f"✓ {city}: {green_pct:.2f}% green pixels")
        
    except Exception as e:
        print(f"✗ {city}: {str(e)}")

print("\n✓ Labels rasterized!")

## 5. Train Random Forest Classifier

Train ML model with 1% pixel sampling for green space detection.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuration
CITY = "London"
SAMPLE_PERCENTAGE = 0.33  # Use 1% of training pixels
OUTPUT_BASE = "results"

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_folder = f"{OUTPUT_BASE}/{CITY}_{timestamp}"
os.makedirs(run_folder, exist_ok=True)

print(f"Training Random Forest for {CITY}")
print(f"Results → {run_folder}")

In [ ]:
# Load satellite data and labels
with rasterio.open(f"processed_stacks/{CITY}_MultiMonth_stack.tif") as src:
    X = src.read()
    profile = src.profile.copy()

with rasterio.open(f"sentinel_data/{CITY}/{CITY}_OSM_labels.tif") as src:
    y = src.read(1)

print(f"Stack: {X.shape}")
print(f"Labels: {y.shape}")

In [ ]:
# Flatten and remove NaN
n_bands, h, w = X.shape
X_flat = X.reshape(n_bands, -1).T
y_flat = y.flatten()

mask = ~np.isnan(X_flat).any(axis=1)
X_flat = X_flat[mask]
y_flat = y_flat[mask]

# Class distribution
unique, counts = np.unique(y_flat, return_counts=True)
for label, count in zip(unique, counts):
    print(f"Class {int(label)}: {count:,} ({100*count/len(y_flat):.2f}%)")

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_flat, y_flat, test_size=0.2, random_state=42, stratify=y_flat
)

# Sample training data
if SAMPLE_PERCENTAGE < 1.0:
    n_samples = int(len(X_train) * SAMPLE_PERCENTAGE)
    X_train, y_train = resample(X_train, y_train, n_samples=n_samples, 
                                random_state=42, stratify=y_train)
    print(f"Sampled: {n_samples:,} pixels ({SAMPLE_PERCENTAGE*100}%)")

print(f"Train: {len(y_train):,} | Test: {len(y_test):,}")

In [ ]:
# Train Random Forest
clf = RandomForestClassifier(
    n_estimators=200, max_depth=25, min_samples_split=10,
    min_samples_leaf=5, class_weight="balanced",
    n_jobs=-1, random_state=42, verbose=1
)

clf.fit(X_train, y_train)
print("\n✓ Training complete!")

In [ ]:
# Predict and evaluate
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["Non-green", "Green"]))

cm = confusion_matrix(y_test, y_pred)
print(f"\nConfusion Matrix:")
print(f"TN: {cm[0,0]:,} | FP: {cm[0,1]:,}")
print(f"FN: {cm[1,0]:,} | TP: {cm[1,1]:,}")
print(f"\nF1 Score (Green): {f1_score(y_test, y_pred, pos_label=1):.3f}")

In [ ]:
# Band importance
months = ['April', 'August', 'November']
bands = ['B02', 'B03', 'B04', 'B08']
band_names = [f"{m}_{b}" for m in months for b in bands]

importance = pd.DataFrame({
    'band': band_names,
    'importance': clf.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 Important Bands:")
print(importance.head(10).to_string(index=False))

In [ ]:
# Generate full prediction map
X_full = X.reshape(n_bands, -1).T
mask_full = ~np.isnan(X_full).any(axis=1)

y_pred_full = np.zeros(X_full.shape[0])
y_pred_full[mask_full] = clf.predict(X_full[mask_full])
y_pred_map = y_pred_full.reshape(h, w)

print("✓ Prediction map created")

In [ ]:
# Save prediction map
pred_profile = profile.copy()
pred_profile.update(dtype='uint8', count=1, compress='lzw')

with rasterio.open(f"{run_folder}/prediction_map.tif", 'w', **pred_profile) as dst:
    dst.write(y_pred_map.astype(np.uint8), 1)

# Save reports
importance.to_csv(f"{run_folder}/feature_importance.csv", index=False)
pd.DataFrame(cm, index=['Actual_0', 'Actual_1'], 
            columns=['Pred_0', 'Pred_1']).to_csv(f"{run_folder}/confusion_matrix.csv")

print(f"\n✓ Results saved to: {run_folder}")

In [ ]:
# Comparison visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# RGB
rgb = X[[6, 5, 4], :, :].transpose(1, 2, 0)
rgb_norm = np.clip(rgb / 3000, 0, 1)
axes[0].imshow(rgb_norm)
axes[0].set_title("RGB (August)", fontsize=14)
axes[0].axis('off')

# Ground truth
axes[1].imshow(y, cmap='RdYlGn', vmin=0, vmax=1)
axes[1].set_title("Ground Truth (OSM)", fontsize=14)
axes[1].axis('off')

# Prediction
axes[2].imshow(y_pred_map, cmap='RdYlGn', vmin=0, vmax=1)
axes[2].set_title("RF Prediction", fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.savefig(f"{run_folder}/comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved")